In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from ResNet import Bottleneck, ResNet, ResNet50

In [2]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

In [3]:
train = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)

trainloader = torch.utils.data.DataLoader(train, batch_size=128, shuffle=True, num_workers=2)

test = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

testloader = torch.utils.data.DataLoader(test, batch_size=128,shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [5]:
PATH = './resNet50_cifar_200_epochs.pth'
net = ResNet50(10)
net.load_state_dict(torch.load(PATH))
net.to('cuda')

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (batch_norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (max_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (batch_norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (batch_norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
      (batch_norm3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (i_downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(256, eps=1e-05, m

In [6]:
# net = ResNet50(10).to('cuda')

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.1, patience=5)

In [7]:
from torchsummary import summary
summary(net, (3,32, 32),64,device="cuda")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [64, 64, 16, 16]           9,408
       BatchNorm2d-2           [64, 64, 16, 16]             128
              ReLU-3           [64, 64, 16, 16]               0
         MaxPool2d-4             [64, 64, 8, 8]               0
            Conv2d-5             [64, 64, 8, 8]           4,160
       BatchNorm2d-6             [64, 64, 8, 8]             128
              ReLU-7             [64, 64, 8, 8]               0
            Conv2d-8             [64, 64, 8, 8]          36,928
       BatchNorm2d-9             [64, 64, 8, 8]             128
             ReLU-10             [64, 64, 8, 8]               0
           Conv2d-11            [64, 256, 8, 8]          16,640
      BatchNorm2d-12            [64, 256, 8, 8]             512
           Conv2d-13            [64, 256, 8, 8]          16,640
      BatchNorm2d-14            [64, 25

In [7]:
EPOCHS = 200
for epoch in range(EPOCHS):
    losses = []
    running_loss = 0
    for i, inp in enumerate(trainloader):
        inputs, labels = inp
        inputs, labels = inputs.to('cuda'), labels.to('cuda')
        optimizer.zero_grad()
    
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if i%100 == 0 and i > 0:
            print(f'Loss [{epoch+1}, {i}](epoch, minibatch): ', running_loss / 100)
            running_loss = 0.0

    avg_loss = sum(losses)/len(losses)
    scheduler.step(avg_loss)
            
print('Training Done')

Loss [1, 100](epoch, minibatch):  8.586315660476684
Loss [1, 200](epoch, minibatch):  2.9610777282714844
Loss [1, 300](epoch, minibatch):  2.4179257237911225
Loss [2, 100](epoch, minibatch):  2.049711730480194
Loss [2, 200](epoch, minibatch):  1.9753414154052735
Loss [2, 300](epoch, minibatch):  1.904314067363739
Loss [3, 100](epoch, minibatch):  1.8436583340168
Loss [3, 200](epoch, minibatch):  1.7955202376842498
Loss [3, 300](epoch, minibatch):  1.7686460900306702
Loss [4, 100](epoch, minibatch):  1.7251257956027986
Loss [4, 200](epoch, minibatch):  1.6807338118553161
Loss [4, 300](epoch, minibatch):  1.6840068590641022
Loss [5, 100](epoch, minibatch):  1.637706241607666
Loss [5, 200](epoch, minibatch):  1.606916286945343
Loss [5, 300](epoch, minibatch):  1.567056783437729
Loss [6, 100](epoch, minibatch):  1.5497723591327668
Loss [6, 200](epoch, minibatch):  1.498753387928009
Loss [6, 300](epoch, minibatch):  1.4916783726215364
Loss [7, 100](epoch, minibatch):  1.4509439957141876
Los

KeyboardInterrupt: 

In [8]:
PATH = './resNet50_cifar.pth'
torch.save(net.state_dict(), PATH)

In [9]:
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to('cuda'), labels.to('cuda')
        outputs = net(images)
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy on 10,000 test images: ', 100*(correct/total), '%')

Accuracy on 10,000 test images:  83.88 %
